In [ ]:
import cv2
import numpy as np
import scipy.fftpack as fftpack
import os
import re

def embed_watermark(image_path, watermark_path, alpha):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded. Check the file path and extension.")
    
    # Load watermark image and convert it to grayscale
    watermark = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    if watermark is None:
        raise ValueError(f"Watermark at path {watermark_path} could not be loaded. Check the file path and extension.")
    
    # Convert image to YCbCr color space
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y_channel, cb, cr = cv2.split(ycbcr_image)
    
    # Apply DCT to the luminance channel
    dct_y = fftpack.dct(fftpack.dct(y_channel.T, norm='ortho').T, norm='ortho')

    # Calculate the size of the high-frequency quadrant
    rows, cols = dct_y.shape
    high_freq_shape = (cols//2, rows//2)  # Note the order of dimensions

    # Resize watermark to match the size of the high-frequency quadrant
    resized_watermark = cv2.resize(watermark, high_freq_shape)

    # Embed watermark into the high-frequency DCT coefficients
    dct_slice = dct_y[rows//2:, cols//2:]  # Taking the bottom-right quarter
    dct_slice += alpha * resized_watermark  # Modify DCT coefficients in place

    # Apply inverse DCT to get the watermarked luminance channel
    y_channel_marked = fftpack.idct(fftpack.idct(dct_y.T, norm='ortho').T, norm='ortho')

    # Ensure the y_channel_marked has the same size and type as original y_channel
    y_channel_marked = cv2.resize(y_channel_marked.astype('float32'), (cb.shape[1], cb.shape[0]))
    y_channel_marked = np.clip(y_channel_marked, 0, 255)  # Ensure values are valid for uint8
    y_channel_marked = y_channel_marked.astype('uint8')  # Convert to uint8

    # Merge channels and convert back to RGB
    watermarked_ycbcr = cv2.merge([y_channel_marked, cb, cr])
    watermarked_image = cv2.cvtColor(watermarked_ycbcr, cv2.COLOR_YCrCb2BGR)
    
    return watermarked_image

def process_images_in_directory(images_dir, watermark_path, alphas):
    # Iterate over all images in the directory
    for filename in os.listdir(images_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            image_number = re.search(r'\d+', filename).group()
            image_path = os.path.join(images_dir, filename)
            for alpha in alphas:
                # Embed the watermark with the given alpha
                watermarked_image = embed_watermark(image_path, watermark_path, alpha)
                # Create directory for alpha if it doesn't exist
                alpha_dir = os.path.join(images_dir, f'alpha_{alpha}')
                os.makedirs(alpha_dir, exist_ok=True)
                # Construct the output filename
                output_filename = f'watermarked_{image_number}_alpha_{alpha}.png'  # Save as .png
                output_path = os.path.join(alpha_dir, output_filename)
                # Save the watermarked image
                cv2.imwrite(output_path, watermarked_image)
                print(f"Saved {output_path}")

# Define the directory and watermark path
images_dir = r'C:\Users\ptcfi\OneDrive\Desktop\Spring 2024\CS700B- Project\AIGI\Images'
watermark_path = r'C:\Users\ptcfi\OneDrive\Desktop\Spring 2024\CS700B- Project\AIGI\watermark_jinal.png'

# Define alpha values
alphas = [0.13,0.15]

# Process all images
process_images_in_directory(images_dir, watermark_path, alphas)
